<a href="https://colab.research.google.com/github/s-c-soma/Deeplearning-CMPE-258/blob/main/Assignment_5/Assignment_5d_XGBoost_End2End_ChicagoTaxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End2End Model Using ChicagoTaxi Dataset and using XGBoost



## Implementation Details and Discussion

* This colab implements the end to end model for Chicago Taxi Dataset using XGBoost

* Here I have followed the steps following reference colab provided by Professor:
> 1. Data Load: 25k Taxi data
> 2. Pre-processing of the data like droping NA values
> 3. Then I have done some feature engineering on the features
> 4. After that I have extracted the features
> 5. Then I have setup the params and model
> 6. After 100 round of training train-rmse:2.37277,test-rmse:3.11873

**Extra Credit**: I have achieved state of art **3.0 rmse accuracy**


# Check if GPU/TPU available

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Apr 24 08:33:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# import libraries

In [ ]:
# load some default Python modules
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

plt.style.use('seaborn-whitegrid')

import warnings
warnings.filterwarnings('ignore')

# Load Data
Download New York City Taxi Fare dataset from Kaggle API 

* Competition link: https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/overview
* Tutorial on Kaggle API: https://towardsdatascience.com/setting-up-kaggle-in-google-colab-ebb281b61463


In [ ]:
!pip install kaggle
!mkdir .kaggle

You should consider upgrading via the '/Users/haley/anaconda3/bin/python -m pip install --upgrade pip' command.


In [ ]:
import json
token = {"username":"","key":""}
with open('/Users/haley/Desktop/258_HW/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!cp /Users/haley/Desktop/258_HW/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/Users/haley/Desktop/258_HW}

- path is now set to: {/Users/haley/Desktop/258_HW}


In [ ]:
!chmod 600 /Users/haley/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c new-york-city-taxi-fare-prediction -p /Users/haley

100%|█████████████████████████████████████▉| 1.56G/1.56G [01:03<00:00, 28.6MB/s]
100%|██████████████████████████████████████| 1.56G/1.56G [01:03<00:00, 26.5MB/s]


# Data Cleaning/Preprocessing

In [ ]:
df = pd.read_csv('new-york-city-taxi-fare-prediction/train.csv')
df.shape

(55423856, 8)

In [ ]:
df_sample = df.sample(25000) # Only use 25k data from the NYC Taxi Dataset

In [ ]:
df_sample.to_csv("NYC_data.csv") # Save sample data for other notebooks

In [ ]:
df_sample

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
6918581,2012-04-20 18:20:00.00000043,9.70,2012-04-20 18:20:00 UTC,-73.957867,40.765155,-73.984122,40.759095,4
6135694,2009-12-19 15:43:21.0000003,4.10,2009-12-19 15:43:21 UTC,-73.954563,40.780773,-73.961841,40.779553,1
38328861,2014-08-17 08:24:36.0000001,6.00,2014-08-17 08:24:36 UTC,-73.992020,40.748710,-74.001072,40.755467,2
28356410,2011-01-01 13:48:15.0000001,7.30,2011-01-01 13:48:15 UTC,-73.973956,40.743322,-73.990257,40.756541,1
8634320,2009-12-15 22:33:21.0000003,6.90,2009-12-15 22:33:21 UTC,-73.979009,40.761543,-73.996857,40.742475,1
...,...,...,...,...,...,...,...,...
46069258,2014-04-02 20:15:05.0000007,21.00,2014-04-02 20:15:05 UTC,0.000000,0.000000,-73.967152,40.608312,1
30358263,2010-06-03 13:18:10.0000002,11.70,2010-06-03 13:18:10 UTC,-74.005335,40.707280,-74.003799,40.748153,1
7321525,2013-07-02 23:58:00.00000039,5.50,2013-07-02 23:58:00 UTC,-73.982965,40.739532,-73.970382,40.756327,5
17786812,2010-04-01 14:55:11.0000002,27.87,2010-04-01 14:55:11 UTC,-73.863547,40.769549,-73.967525,40.805443,2


# Pre Processing

In [ ]:
df_sample = df_sample.dropna()
df_sample = df_sample.drop_duplicates()

In [ ]:
df_sample.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,11.230264,-72.498820,39.953620,-72.456549,39.911780,1.676000
std,9.481905,10.443341,7.071689,10.585145,6.566459,1.304198
min,0.000000,-74.689831,-74.000353,-74.689831,-74.001537,0.000000
25%,6.000000,-73.991920,40.735795,-73.991259,40.734021,1.000000
50%,8.500000,-73.981830,40.752976,-73.980136,40.753132,1.000000
75%,12.500000,-73.967197,40.767344,-73.963265,40.768369,2.000000
max,300.000000,40.769807,482.500000,40.774482,404.216667,6.000000


# Feature Engineering

In [ ]:
def prepare_time_features(df):
    df['pickup_datetime'] = df['pickup_datetime'].str.slice(0, 16)
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    df['hour_of_day'] = df.pickup_datetime.dt.hour
    df['month'] = df.pickup_datetime.dt.month
    df["year"] = df.pickup_datetime.dt.year
    df["weekday"] = df.pickup_datetime.dt.weekday
    return df

In [ ]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [ ]:
def transform(data):
    # Distances to nearby airports, 
    jfk = (-73.7781, 40.6413)
    ewr = (-74.1745, 40.6895)
    lgr = (-73.8740, 40.7769)

    data['pickup_distance_to_jfk'] = distance(jfk[1], jfk[0],
                                         data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_jfk'] = distance(jfk[1], jfk[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_ewr'] = distance(ewr[1], ewr[0], 
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_ewr'] = distance(ewr[1], ewr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_lgr'] = distance(lgr[1], lgr[0],
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_lgr'] = distance(lgr[1], lgr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    
    return data

In [ ]:
df_sample = prepare_time_features(df_sample)
df_sample['distance_miles'] = distance(df_sample.pickup_latitude, df_sample.pickup_longitude,df_sample.dropoff_latitude, df_sample.dropoff_longitude)

In [ ]:
df_sample = transform(df_sample)
df_train = df_sample.drop(columns=['key','pickup_datetime'])
df_train.head()

,Unnamed: 0,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour_of_day,month,year,weekday,distance_miles,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr
0,6918581,9.7,-73.957867,40.765155,-73.984122,40.759095,4,18,4,2012,4,1.436387,12.723780,13.516804,12.489521,11.067880,4.462835,5.892372
1,6135694,4.1,-73.954563,40.780773,-73.961841,40.779553,1,15,12,2009,5,0.389996,13.352104,13.559267,13.128499,12.754368,4.223540,4.599535
2,38328861,6.0,-73.992020,40.748710,-74.001072,40.755467,2,8,8,2014,6,0.665161,13.440951,14.094173,10.394813,10.161043,6.476132,6.812570
3,28356410,7.3,-73.973956,40.743322,-73.990257,40.756541,1,13,1,2011,5,1.249886,12.448576,13.671388,11.141253,10.702055,5.722552,6.244181
4,8634320,6.9,-73.979009,40.761543,-73.996857,40.742475,1,22,12,2009,1,1.615064,13.407936,13.424094,11.382396,9.997209,5.596372,6.855570


In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('fare_amount', axis=1),
                                                    df_train['fare_amount'], test_size=0.3, random_state = 42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(17500, 17)
(17500,)
(7500, 17)
(7500,)


# XGBoost Model

## Param Setup

In [ ]:
import xgboost as xgb
params = {
    'max_depth': 8,
    'gamma' :0,
    'eta':.05, 
    'subsample': 1,
    'colsample_bytree': 0.9, 
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 0
}

## Model and Training

In [ ]:
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=5000, 
                    early_stopping_rounds=10,evals=[(matrix_train, 'train'),(matrix_test,'test')])
    return model
model = XGBmodel(X_train,X_test,y_train,y_test,params)

[02:25:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:13.7565	test-rmse:13.4699
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 10 rounds.
[1]	train-rmse:13.143	test-rmse:12.8597
[2]	train-rmse:12.5592	test-rmse:12.2869
[3]	train-rmse:12.0345	test-rmse:11.7714
[4]	train-rmse:11.5085	test-rmse:11.251
[5]	train-rmse:11.0114	test-rmse:10.7601
[6]	train-rmse:10.5403	test-rmse:10.2984
[7]	train-rmse:10.0956	test-rmse:9.86193
[8]	train-rmse:9.67346	test-rmse:9.45222
[9]	train-rmse:9.27329	test-rmse:9.06563
[10]	train-rmse:8.89625	test-rmse:8.70027
[11]	train-rmse:8.53966	test-rmse:8.35654
[12]	train-rmse:8.20253	test-rmse:8.03827
[13]	train-rmse:7.88486	test-rmse:7.73461
[14]	train-rmse:7.60156	test-rmse:7.46295
[15]	train-rmse:7.31514	test-rmse:7.19613
[16]	train-rmse:7.0461	test-rmse:6.945
[17]	train-rmse:6.79351	test-rm